In [ ]:
pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import datetime

In [ ]:
etfs = fdr.StockListing('ETF/KR')

In [ ]:
filter_etfs = etfs['Volume'] > 100000
filtered_etfs = etfs[filter_etfs]
filtered_etfs

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
0,459580,6,KODEX CD금리액티브(합성),1052280,2,100,0.01,1052317.0,0.8815,248927,261940,87296
1,357870,6,TIGER CD금리투자KIS(합성),55155,2,5,0.01,55153.0,0.8686,129758,7156,65888
2,069500,1,KODEX 200,34590,2,25,0.07,34571.0,-7.2232,3698700,128100,59270
3,360750,4,TIGER 미국S&P500,20155,2,130,0.65,20149.0,7.6081,10966446,219954,49732
5,133690,4,TIGER 미국나스닥100,125380,2,1230,0.99,125318.0,6.6283,197078,24614,38905
...,...,...,...,...,...,...,...,...,...,...,...,...
725,494340,4,ACE 글로벌AI맞춤형반도체,9800,5,-10,-0.10,9825.0,NaN,114876,1119,78
748,284980,2,RISE 200금융,12335,2,425,3.57,12362.0,11.8110,101289,1257,72
779,485810,4,TIMEFOLIO 글로벌안티에이징바이오액티브,9950,5,-65,-0.65,10018.0,0.8616,139637,1397,65
783,253230,3,KOSEF 200선물인버스2X,2265,3,0,0.00,2262.0,12.1287,142036,319,64


In [ ]:
#filtered_etfs.to_excel('filtered_etfs.xlsx')

In [ ]:
# 금리 ETF 필터링 (이름에 '금리' 포함)
interest_rate_etfs = filtered_etfs[filtered_etfs['Name'].str.contains('금리|머니')]
# 채권형 ETF 필터링 (이름에 '채권' 포함)
bond_etfs =filtered_etfs[filtered_etfs['Name'].str.contains('채')&~filtered_etfs['Name'].str.contains('테슬라|S&P500|삼성전자')]
# 주식형 ETF 필터링 (ETF 이름이나 설명에서 특정 주식 지수를 기준으로 필터링)
stock_etfs = filtered_etfs[filtered_etfs['Name'].str.contains('S&P500|나스닥|코스닥|다우존스|코스피|인도|항셍|200') & ~filtered_etfs['Name'].str.contains('커버드콜')]
# 커버드콜 ETF 필터링 (ETF 이름이나 설명에서 커버드 기준으로 필터링)
coveredcall_etfs = filtered_etfs[filtered_etfs['Name'].str.contains('커버드콜')]

industry_etfs = filtered_etfs[
    ~filtered_etfs.index.isin(interest_rate_etfs.index) &
    ~filtered_etfs.index.isin(bond_etfs.index) &
    ~filtered_etfs.index.isin(stock_etfs.index) &
    ~filtered_etfs.index.isin(coveredcall_etfs.index)
]

In [ ]:
tiger_etf=filtered_etfs[filtered_etfs['Name'].str.contains('TIGER')]
kodex_etf=filtered_etfs[filtered_etfs['Name'].str.contains('KODEX')]

In [ ]:
tiger_etf.describe()

,Category,Price,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
count,62.000000,6.200000e+01,62.000000,62.000000,6.200000e+01,58.000000,6.200000e+01,62.000000,62.000000
mean,3.532258,3.096306e+04,79.274194,0.467258,3.097748e+04,6.261126,9.407775e+05,12837.419355,6645.080645
std,1.544082,1.297794e+05,219.531630,1.540640,1.297743e+05,18.461401,1.865213e+06,32001.293038,12224.737144
min,1.000000,2.320000e+03,-385.000000,-3.150000,2.323000e+03,-36.456000,1.005750e+05,330.000000,61.000000
25%,2.000000,7.058750e+03,-32.500000,-0.585000,7.063500e+03,-5.075100,1.707940e+05,2016.000000,800.750000
50%,4.000000,1.036000e+04,55.000000,0.515000,1.038700e+04,4.805450,3.151530e+05,4456.500000,1713.500000
75%,4.000000,1.284500e+04,108.750000,0.970000,1.285150e+04,9.039350,1.006645e+06,9305.750000,6353.500000
max,7.000000,1.026675e+06,1230.000000,6.440000,1.026671e+06,69.031300,1.096645e+07,219954.000000,65888.000000


In [ ]:
codes=tiger_etf['Symbol'].tolist()
start_date = datetime.date(2014, 1, 1)
end_date = datetime.date(2024, 9, 30)

In [ ]:
stock_data = {}

# 종가 데이터를 불러오는 for 루프
for code in codes:
    # 각 종목의 2024년부터 현재까지의 주가 데이터 불러오기
    df = fdr.DataReader(code, start_date, end_date)

    # 월별 마지막 날의 종가만 선택
    monthly_close = df['Close'].resample('ME').last()

    # 딕셔너리에 저장
    stock_data[code] = monthly_close

# 모든 종목의 월별 종가 데이터를 하나의 DataFrame으로 결합
result = pd.DataFrame(stock_data)
result

,357870,360750,133690,381170,381180,102110,371460,458730,453540,305540,...,491010,365000,396520,139220,488080,494670,139270,139250,250780,139240
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-31,NaN,NaN,16303,NaN,NaN,20605,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3388,NaN,NaN,4975,8775,NaN,7131
2014-02-28,NaN,NaN,16989,NaN,NaN,20929,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3576,NaN,NaN,4933,8896,NaN,6965
2014-03-31,NaN,NaN,16546,NaN,NaN,20928,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3495,NaN,NaN,4881,8613,NaN,7165
2014-04-30,NaN,NaN,15858,NaN,NaN,20715,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3509,NaN,NaN,4747,8475,NaN,7132
2014-05-31,NaN,NaN,16416,NaN,NaN,21181,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3388,NaN,NaN,4799,8226,NaN,7365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,54395.0,18007.0,113936,19310.0,18880.0,35903,7890.0,11214.0,52795.0,21515.0,...,NaN,3725.0,5170.0,3045,NaN,NaN,7887,12347,3735.0,8235
2024-06-30,54540.0,18830.0,121793,21380.0,19897.0,38478,7190.0,11289.0,53050.0,21270.0,...,NaN,3850.0,5250.0,3040,NaN,NaN,8072,12382,3655.0,8235
2024-07-31,54700.0,18765.0,116785,20055.0,18530.0,38185,7075.0,12098.0,53275.0,19080.0,...,NaN,3580.0,5385.0,3440,9120.0,NaN,8805,11400,3715.0,7375


In [ ]:
#sorted_result

In [ ]:
# 월별 수익률 계산
month_end_ret = result.pct_change()
month_end_ret_real=month_end_ret.iloc[1:,:]

In [ ]:
month_end_ret_real.iloc[:7,:7]

,357870,360750,133690,381170,381180,102110,371460
Date,,,,,,,
2014-02-28,NaN,NaN,0.042078,NaN,NaN,0.015724,NaN
2014-03-31,NaN,NaN,-0.026076,NaN,NaN,-0.000048,NaN
2014-04-30,NaN,NaN,-0.041581,NaN,NaN,-0.010178,NaN
2014-05-31,NaN,NaN,0.035187,NaN,NaN,0.022496,NaN
2014-06-30,NaN,NaN,0.024184,NaN,NaN,-0.004296,NaN
2014-07-31,NaN,NaN,0.047463,NaN,NaN,0.040683,NaN
2014-08-31,NaN,NaN,0.013117,NaN,NaN,-0.018908,NaN


In [ ]:
def momentum_strategy(df, M, N):
    results = []
    total_returns = []  # 총 수익률을 저장할 리스트

    for start_month in range(len(df) - M - N + 1):
        # 룩백 기간 동안의 수익률 누적 계산
        lookback_period = df.iloc[start_month : start_month + M]
        cumulative_returns = lookback_period.add(1).prod() - 1

        # 상위 3개 종목 (롱) 및 하위 3개 종목 (숏) 선정
        long_assets = cumulative_returns.nlargest(3).index
        short_assets = cumulative_returns.nsmallest(3).index

        # 보유 기간 동안의 수익률 계산
        holding_period = df.iloc[start_month + M : start_month + M + N]
        long_return = holding_period[long_assets].add(1).prod(axis=0).mean() - 1
        short_return = holding_period[short_assets].add(1).prod(axis=0).mean() - 1

        # 총 수익률 계산 및 리스트에 추가
        total_return = long_return - short_return
        total_returns.append(total_return)

        # 결과 저장
        results.append({
            'Lookback_Date': df.index[start_month + M - 1],
            'Pfo_Date': df.index[start_month + M + N - 1],  # 해당 보유 기간의 마지막 달
            'Long Assets': list(long_assets),
            'Short Assets': list(short_assets),
            'Long Return': long_return,
            'Short Return': short_return,
            'Total Return': total_return,
        })

    # DataFrame으로 변환
    result_df = pd.DataFrame(results)

    # Tranche Return 열 추가
    result_df['Tranche Return'] = pd.Series(total_returns).rolling(window=N, min_periods=1).mean()

    return result_df

In [ ]:
# 데이터프레임과 M, N 값 설정 예시
# monthly_returns = pd.DataFrame()  # 실제 데이터프레임 필요
M = 7  # 룩백 기간
N = 3  # 보유 기간
result_df = momentum_strategy(month_end_ret_real, M, N)
result_df

,Lookback_Date,Pfo_Date,Long Assets,Short Assets,Long Return,Short Return,Total Return,Tranche Return
0,2014-08-31,2014-11-30,"[139240, 139270, 133690]","[139230, 143860, 139250]",0.018379,-0.065344,0.083723,0.083723
1,2014-09-30,2014-12-31,"[139240, 192090, 133690]","[139230, 139250, 139220]",0.166841,-0.112937,0.279778,0.181751
2,2014-10-31,2015-01-31,"[192090, 133690, 139270]","[139230, 139250, 139220]",0.130737,-0.010808,0.141545,0.168349
3,2014-11-30,2015-02-28,"[192090, 204480, 133690]","[139250, 139230, 139220]",0.271379,0.062660,0.208719,0.210014
4,2014-12-31,2015-03-31,"[204480, 192090, 133690]","[139220, 139230, 139250]",0.222293,0.089771,0.132522,0.160929
...,...,...,...,...,...,...,...,...
114,2024-02-29,2024-05-31,"[139270, 466940, 381180]","[412570, 456680, 438320]",0.096626,-0.045058,0.141683,0.090133
115,2024-03-31,2024-06-30,"[396500, 381180, 466940]","[412570, 456680, 438320]",0.091518,-0.200588,0.292106,0.083642
116,2024-04-30,2024-07-31,"[381180, 396500, 466940]","[412570, 252710, 456680]",0.063773,-0.250382,0.314155,0.249315
117,2024-05-31,2024-08-31,"[381180, 139230, 396500]","[412570, 252710, 456680]",-0.029681,-0.134577,0.104896,0.237052


In [ ]:
def match_names(row):
    long_names = [filtered_etfs.loc[filtered_etfs['Symbol'] == symbol, 'Name'].values[0] for symbol in row['Long Assets']]
    short_names = [filtered_etfs.loc[filtered_etfs['Symbol'] == symbol, 'Name'].values[0] for symbol in row['Short Assets']]
    return pd.Series(long_names + short_names, index=['Long 1', 'Long 2', 'Long 3', 'Short 1', 'Short 2', 'Short 3'])

# 이름 매칭 결과를 새로운 데이터프레임으로 생성
matched_df = result_df.apply(match_names, axis=1)
matched_df.index = result_df['Lookback_Date']  # Lookback_Date를 인덱스로 설정

# 결과 출력
matched_df

,Long 1,Long 2,Long 3,Short 1,Short 2,Short 3
Lookback_Date,,,,,,
2014-08-31,TIGER 200 철강소재,TIGER 200 금융,TIGER 미국나스닥100,TIGER 200 중공업,TIGER 헬스케어,TIGER 200 에너지화학
2014-09-30,TIGER 200 철강소재,TIGER 차이나CSI300,TIGER 미국나스닥100,TIGER 200 중공업,TIGER 200 에너지화학,TIGER 200 건설
2014-10-31,TIGER 차이나CSI300,TIGER 미국나스닥100,TIGER 200 금융,TIGER 200 중공업,TIGER 200 에너지화학,TIGER 200 건설
2014-11-30,TIGER 차이나CSI300,TIGER 차이나CSI300레버리지(합성),TIGER 미국나스닥100,TIGER 200 에너지화학,TIGER 200 중공업,TIGER 200 건설
2014-12-31,TIGER 차이나CSI300레버리지(합성),TIGER 차이나CSI300,TIGER 미국나스닥100,TIGER 200 건설,TIGER 200 중공업,TIGER 200 에너지화학
...,...,...,...,...,...,...
2024-02-29,TIGER 200 금융,TIGER 은행고배당플러스TOP10,TIGER 미국필라델피아반도체나스닥,TIGER 2차전지TOP10레버리지,TIGER 차이나전기차레버리지(합성),TIGER 차이나항셍테크레버리지(합성 H)
2024-03-31,TIGER Fn반도체TOP10,TIGER 미국필라델피아반도체나스닥,TIGER 은행고배당플러스TOP10,TIGER 2차전지TOP10레버리지,TIGER 차이나전기차레버리지(합성),TIGER 차이나항셍테크레버리지(합성 H)
2024-04-30,TIGER 미국필라델피아반도체나스닥,TIGER Fn반도체TOP10,TIGER 은행고배당플러스TOP10,TIGER 2차전지TOP10레버리지,TIGER 200선물인버스2X,TIGER 차이나전기차레버리지(합성)


In [ ]:
# 트렌치 리턴의 개수와 평균을 기준으로 최적의 M, N 조합을 찾는 함수
def find_best_momentum_strategy(df):
    best_count_M, best_count_N = None, None  # Tranche Return >= 0 개수가 가장 많은 M, N 조합
    best_avg_M, best_avg_N = None, None      # Tranche Return 평균이 가장 높은 M, N 조합
    max_positive_count = 0
    max_avg_return = float('-inf')

    for M in range(1, 10):  # M 값을 1부터 9까지
        for N in range(1, 4):  # N 값을 1부터 3까지
            # momentum_strategy 함수 실행
            result_df = momentum_strategy(df, M, N)

            # Tranche Return 값이 0 이상인 개수 계산
            positive_count = (result_df['Tranche Return'] >= 0).sum()

            # Tranche Return의 평균 계산
            avg_return = result_df['Tranche Return'].mean()

            # Tranche Return >= 0인 개수가 가장 많은 경우 갱신
            if positive_count > max_positive_count:
                max_positive_count = positive_count
                best_count_M, best_count_N = M, N

            # Tranche Return의 평균이 가장 높은 경우 갱신
            if avg_return > max_avg_return:
                max_avg_return = avg_return
                best_avg_M, best_avg_N = M, N

    return (best_count_M, best_count_N, max_positive_count), (best_avg_M, best_avg_N, max_avg_return)

# 예제 실행 (monthly_returns 데이터를 사용해야 함)
# M = 1~9, N = 1~3의 조합 중에서 가장 많은 Tranche Return >= 0인 조합 및 평균이 가장 높은 조합 찾기
best_count_result, best_avg_result = find_best_momentum_strategy(month_end_ret_real)

print(f"Tranche Return >= 0인 개수가 가장 많은 조합: M = {best_count_result[0]}, N = {best_count_result[1]}, 개수 = {best_count_result[2]}")
print(f"Tranche Return 평균이 가장 높은 조합: M = {best_avg_result[0]}, N = {best_avg_result[1]}, 평균 = {best_avg_result[2]}")


Tranche Return >= 0인 개수가 가장 많은 조합: M = 7, N = 3, 개수 = 77
Tranche Return 평균이 가장 높은 조합: M = 7, N = 3, 평균 = 0.07569476562751282


In [ ]:
result_df['Tranche Return'].describe()

,Tranche Return
count,119.000000
mean,0.063719
std,0.159347
min,-0.395867
25%,-0.020766
50%,0.059600
75%,0.161414
max,0.504993
